## Tabular analysis

This notebook loads a tabular dataset and a pretrained model and computes Qinv and Qbas for all possible rankings.

In [1]:
# Import the necessary libraries
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))
import xai_faithfulness_experiments_lib_edits as fl

import numpy as np

/home/eirasf/miniconda3/envs/xai-anna/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Avila dataset
DATASET = 'avila'
DATASET_PATH = os.path.join(PROJ_DIR,'assets', 'data', f'{DATASET}.npz')
MODEL_PATH = os.path.join(PROJ_DIR,'assets', 'models', f'{DATASET}-mlp.pth')

In [3]:
# Load dataset
file_data = np.load(DATASET_PATH)
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

# Load model
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MODEL_NEURONS = 100
MODEL_EPOCHS= 2000
MODEL_LR = 1.0e-1
MODEL_LABEL_NUM = len(np.unique(y_train))

class MLP(torch.nn.Module):
    def __init__(self, n_neurons):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[1], n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, MODEL_LABEL_NUM)
        self.ac2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        logits = self.fc2(x)
        x = self.ac2(logits)
        return x

network = MLP(MODEL_NEURONS)
network.load_state_dict(torch.load(MODEL_PATH))
network.eval()
network.to(device)

MLP(
  (fc1): Linear(in_features=10, out_features=100, bias=True)
  (ac1): Sigmoid()
  (fc2): Linear(in_features=100, out_features=12, bias=True)
  (ac2): Softmax(dim=1)
)

In [4]:
import itertools

NUM_VARS = x_train.shape[1]
print(NUM_VARS)

permutations = list(itertools.permutations(range(NUM_VARS)))
all_rankings = np.array(permutations) / (NUM_VARS - 1)


10


In [5]:
from tqdm import tqdm

SAMPLE_NUM = 250 # Select one of the training examples in the dataset to be explained

num_rankings = all_rankings.shape[0]
row = torch.tensor(np.float32(x_train[SAMPLE_NUM])).to(device)
label = torch.tensor(y_train[SAMPLE_NUM]).to(device)

# All of these measures will be stored
suffixes = ['', '_inv', '_bas']
size1_prefixes = ['mean', 'at_first_argmax', 'auc']
sizeNUM_SAMPLES_prefixes = ['output_curve', 'is_hit_curve']
keys = ['ranking']
for p in size1_prefixes+sizeNUM_SAMPLES_prefixes:
    for s in suffixes:
        keys.append(p+s)

# Dict to store all results
all_measures = {}
# Initialize all np arrays to speed up the process
for k in size1_prefixes:
    for s in suffixes:
        all_measures[k+s] = np.zeros((num_rankings, 1), dtype=np.float32)

for k in sizeNUM_SAMPLES_prefixes:
    for s in suffixes:
        all_measures[k+s] = np.zeros((num_rankings, fl.NUM_SAMPLES), dtype=np.float32 if 'is_hit' in k else bool)
all_measures['ranking'] = np.zeros((num_rankings, NUM_VARS), dtype=np.float32)

# Compute the results for each possible ranking
for i in tqdm(range(num_rankings)):
    #TODO - Add several samples for qbas instead of a single one
    measures = fl.get_measures_for_ranking(row, torch.tensor(all_rankings[i]).to(device), label, network, num_samples=fl.NUM_SAMPLES, with_inverse=True, with_random=True)
    measures['ranking'] = all_rankings[i]
    # Save all results for this rankings to the i-th position
    for k in keys:
        all_measures[k][i] = measures[k]
    #TODO - For each ranking, retrieve and store Quantus' faithfulness metrics
    break
    
#left_out = all_rankings.shape[0] % BATCH_SIZE
#if left_out > 0:
#    print(all_rankings.shape[0] - left_out, all_rankings.shape[0])

  0%|          | 0/3628800 [00:00<?, ?it/s]


In [14]:
import quantus

metric = quantus.MaxSensitivity(nr_samples=10,
                                lower_bound=0.2)

x_batch = torch.unsqueeze(row, dim=0)
y_batch = torch.unsqueeze(label, dim=0)
a_batch = np.expand_dims(all_measures['ranking'][0],0)

print(x_batch.shape)
print(y_batch.shape)
print(network(x_batch).shape)
print(a_batch.shape)
# TODO - Necessary reshapes
scores = metric(
    model=network,
    x_batch=x_batch,
    y_batch=y_batch,
    a_batch=a_batch,
    device=device
)

Warnings and information:
 (1) The Max Sensitivity metric is likely to be sensitive to the choice of amount of noise added 'lower_bound' and 'upper_bound', the number of samples iterated over 'nr_samples', the function to perturb the input 'perturb_func', the similarity metric 'similarity_func' as well as norm calculations on the numerator and denominator of the sensitivity equation i.e., 'norm_numerator' and 'norm_denominator'.  
 (2) If attributions are normalised or their absolute values are taken it may destroy or skew information in the explanation and as a result, affect the overall evaluation outcome.
 (3) Make sure to validate the choices for hyperparameters of the metric (by calling .get_params of the metric instance).
 (4) For further information, see original publication: Yeh, Chih-Kuan, et al. 'On the (in) fidelity and sensitivity for explanations.' arXiv preprint arXiv:1901.09392 (2019).
 (5) To disable these warnings set 'disable_warnings' = True when initialising the met

AssertionError: Make sure 'explain_func' is a Callable that takes model, x_batch, y_batch and **kwargs as arguments.

In [30]:
np.savez(os.path.join(PROJ_DIR, 'results', f'{DATASET}_{SAMPLE_NUM}_measures.npz'), \
         row=row.to('cpu').numpy(), \
         label=label.to('cpu').numpy(), \
         rankings=all_measures['ranking'], \
         qmeans=all_measures['mean'], \
         qmean_invs=all_measures['mean_inv'], \
         qmean_bas=all_measures['mean_bas'], \
         qargmaxs=all_measures['at_first_argmax'], \
         qargmax_invs=all_measures['at_first_argmax_inv'], \
         qargmax_bas=all_measures['at_first_argmax_bas'], \
         qaucs=all_measures['auc'], \
         qauc_invs=all_measures['auc_inv'], \
         qauc_bas=all_measures['auc_bas'], \
         output_curves=all_measures['output_curve'], \
         is_hit_curves=all_measures['is_hit_curve'], \
         output_curves_inv=all_measures['output_curve_inv'], \
         is_hit_curves_inv=all_measures['is_hit_curve_inv'], \
         output_curves_bas=all_measures['output_curve_bas'], \
         is_hit_curves_bas=all_measures['is_hit_curve_bas'])